# Envoke API Implementation 
Uses a combination of the Contacts and Reporting APIs. Contacts will be used to gather lists of subscribed contacts daily. This will allow for a record of all contacts at any given time, as well as a count of contacts on a daily basis. 

Reporting API is still in beta mode, which means a lot of things are rough around the edges still. Returns stats about: 
- Sends 
- Opens 
- Clicks 
- Bounces 
- Revoked

In [ ]:
import requests 
from requests.auth import HTTPBasicAuth
import os
import json 
import pandas as pd 
import datetime as dt 
from dotenv import load_dotenv, dotenv_values
import time
import numpy as np 

In [ ]:
load_dotenv() 
basic = HTTPBasicAuth(os.getenv("USERNAME"), os.getenv("PASSWORD"))
run_date = str(dt.date.today() - dt.timedelta(days = 1))

## Functions 
- Left: For the dates in the dataset. They are formatted strangely for clients, leading to the creation of this function. Imitates Left in DAX
- Mid: Similar to left, except it allows me to choose any chunk within the string. Some mental math has to be done to get the positions right
- Expander: There are more attributes for each contact stored within a list. This list needs to be expanded to get those individual attributes. This function allows you to search  a list for a specific phrase + return the response associated with said phrase. 

In [ ]:
def left(s, amount): 
    return s[:amount]

def mid(s, offset, amount): 
    return s[offset:offset+amount]

def expander(list, phrase, offset=2, amount=2): 
    detection = [s.find(phrase) > 0 for s in list]

    try:
        detection = detection.index(True)
        resp = list[detection]
        resp = mid(resp, offset, amount)
        return resp
    except ValueError as v: 
        return np.nan
    

## Email Metrics for Previous Day

In [ ]:
def get_email_metrics(run_date): 

        url = f"https://e1.envoke.com/v1/reports/emailActivityMetrics?start_date={run_date}&end_date={run_date}"
        params = {'filter[sent_date]': [f"{run_date}"]}
        payload = {}
        headers = {}


        try: 
            resp = requests.get(url=url, headers=headers, data=payload, auth=basic, params=params) # sending request to API using url, filter, and authorization 

            if resp.status_code == 200: # if response is successful, turn the json file into a df for future use
                
                resp_df = pd.DataFrame(resp.json()) # conversion into dataframe
                resp_df = resp_df[resp_df['sent_date'] == run_date] # additional filtering required at the moment, for some reason the date filter doesn't work.
                
                
                if len(resp_df['message_name']) > 0:
                    old_message_stats = pd.read_csv('envoke/Messages/aggregate_message_metrics.csv')
                    resp_df.to_csv(f'envoke/Messages/message_stats_{run_date}.csv', index=False)
                    new_aggregate = pd.concat([old_message_stats, resp_df])
                    new_aggregate.to_csv("envoke/Messages/aggregate_message_metrics.csv", index=False)
                    return resp_df
                else: 
                    print(f"There have been no messages sent using Envoke on {run_date}")

            else: 
                print(f"Initial request failed due to error code {resp.status_code}")

        except Exception as e: 
            print(f"Error occurred during the request: {e}")

        except requests.exceptions.RequestException as e: # this will handle specific requests exceptions, which can be informative
            print(f"An unexpected error occurred during the request: {e}")

get_email_metrics(run_date=run_date)


There have been no messages sent using Envoke on 2025-04-15


## Active/Subscribed Contacts 
This refers to clients with a constent status that is one of the following: 
- Implied - No Expiry 
- Express


In [ ]:
def get_active_contacts(status_filters=["Implied - No Expiry","Express"], skip=0):
    resp_df = pd.DataFrame()
    check = 1
    all_results = []
    url = f"https://e1.envoke.com/v1/contacts?skip={skip}&limit=100"
    params = {'filter[consent_status]': status_filters}
    payload = {}
    headers = {}

    # need to add a filter for people with status: 'Implied - No Expiry', 'Express' to prevent fetching ALL contacts in the database. 
    try: 
        response = requests.get(url, headers=headers, data=payload, auth=basic, params=params)

        if response.status_code == 200: 
            response_df = pd.DataFrame(response.json()) # convert initial response to dataframe
            all_results.append(response.json()) # append results to list for future concatenation
            check = len(response.json()) # check the length of the response to the request
        else: 
            print(f"Initial request failed with status code {response.status_code}")
            check = 0 
    except Exception as e: 
        print(f"Error occurred during initial request: {e}")
        check = 0 # stop loop if there is request error 

    while check > 0: 
        skip += 100 # adding 100 to the skip number as you are limited to a maximum of 100 contacts per request
        url = f"https://e1.envoke.com/v1/contacts?skip={skip}&limit=100"
        print(f"Skipping {skip-100} contacts.")

        try:
            response = requests.get(url, headers=headers, data=payload, auth=basic, params=params) # send next request 

            if response.status_code == 200: 
                results = response.json() # get the json response 
                all_results.append(results)
                check = len(results)


                time.sleep(1) 
            else: 
                print(f"Request failed with code {response.status_code}")
                check = 0 # stop loop 
        except Exception as e: 
            print(f"Error occurred during the request: {e}")
            check = 0 
    current_contacts = pd.concat([pd.DataFrame(result) for result in all_results], ignore_index=True)

    current_contacts['full_name'] = current_contacts['first_name'] + ' ' + current_contacts['last_name'] # full name to make things easier 
    current_contacts['full_name'] = current_contacts['full_name'].replace(r"^\s*$", np.nan, regex=True)
    current_contacts['date_created'] = current_contacts['date_created'].apply(lambda x: left(x, 10))
    current_contacts['current_director'] = current_contacts['interests'].apply(lambda x: expander(x, "currently a director", amount=3))
    current_contacts['been_director'] = current_contacts['interests'].apply(lambda x: expander(x, "been a director",  amount=3))
    current_contacts['is_condo_owner'] = current_contacts['interests'].apply(lambda x: expander(x, "a condo owner", amount=3))
    current_contacts['referral'] = current_contacts['interests'].apply(lambda x: True if expander(x, "Referral") == "Ref" else False)
    current_contacts["full_name"] = current_contacts["full_name"].str.title()
    return current_contacts

current_contacts = get_active_contacts()

Skipping 0 contacts.
Skipping 100 contacts.
........
Skipping 100000 contacts.
Skipping 100100 contacts.


In [ ]:
d = {'date': [run_date], 
     'total_active_contacts': [len(current_contacts['id'])]}

master_contact_list_addon = pd.DataFrame(d)

In [ ]:

current_contacts = current_contacts.drop(columns=['custom_fields','interests', 'autoresponders'])

In [ ]:
current_contacts.to_csv(f"envoke/Contacts/active_contacts.csv", index=False) # need to maintain most current list of contacts as dataset for contact demographics 
master_contact_list_addon.to_csv(f"envoke/Contacts/master_contact_list{run_date}.csv", index=False)